In [1]:
import datetime
import glob
import pandas as pd
import xarray as xr

In [2]:
obs_files = glob.glob('/nfs/a68/earlacoa/china_measurements_corrected/*.nc')

In [3]:
df_obs = pd.DataFrame({
    'station_id': [],
    'station_lat': [],
    'station_lon': [],
    'name': [],
    'prefecture': [],
    'o3_6mDM8h_ppb': [],
    'datetime': []
})

In [4]:
def create_o3_metric(obs_file, df_obs):
    ds = xr.open_dataset(obs_file)
    o3 = ds['O3']
    years = ['2015', '2016', '2017', '2018', '2019']    
 
    for year in years:
        o3_year = o3.sel(time=year)
        
        # first: 24, 8-hour, rolling mean, O3 concentrations
        o3_6mDM8h_8hrrollingmean = o3_year.rolling(time=8).construct('window').mean('window')

        # second: find the max of these each day (daily maximum, 8-hour)
        o3_6mDM8h_dailymax = o3_6mDM8h_8hrrollingmean.sortby('time').resample(time='24H').max()

        # third: 6-month mean - to account for different times when seasonal maximums e.g. different hemispheres
        o3_6mDM8h_6monthmean = o3_6mDM8h_dailymax.resample(time='6M').mean()

        # fourth: maximum of these
        o3_6mDM8h = o3_6mDM8h_6monthmean.max(dim='time')

        # convert units from µg m-3 to ppb - using 1 ppb = 1.9957 µg m-3
        o3_6mDM8h_ppb = o3_6mDM8h.values / 1.9957

        df = pd.DataFrame({
            'station_id': [ds.station],
            'station_lat': [ds.lat],
            'station_lon': [ds.lon],
            'name': [ds.name],
            'prefecture': [ds.city],
            'o3_6mDM8h_ppb': [o3_6mDM8h_ppb],
            'datetime': [pd.to_datetime(datetime.datetime(int(year), 1, 1))]
        })

        df_obs = df_obs.append(df)
    
    ds.close()
    
    return df_obs

In [ ]:
for obs_file in obs_files:
    df_obs = create_o3_metric(obs_file, df_obs)
    
df_obs = df_obs.set_index('datetime').sort_index()

/nfs/b0122/Users/earlacoa/miniconda3/envs/python3/lib/python3.8/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/nfs/b0122/Users/earlacoa/miniconda3/envs/python3/lib/python3.8/site-packages/xarray/core/common.py:1105: FutureWarning: 'base' in .resample() and in Grouper() is deprecated.
The new arguments that you should use are 'offset' or 'origin'.

>>> df.resample(freq="3s", base=2)

becomes:

>>> df.resample(freq="3s", offset="2s")

  grouper = pd.Grouper(
/nfs/b0122/Users/earlacoa/miniconda3/envs/python3/lib/python3.8/site-packages/xarray/core/nputils.py:215: RuntimeWarning: All-NaN slice encountered
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/nfs/b0122/Users/earlacoa/miniconda3/envs/python3/lib/python3.8/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/nfs/b0122/Users/earlacoa/miniconda3/envs/python3/lib/python3

In [ ]:
df_obs.to_csv('/nfs/a68/earlacoa/china_measurements_corrected/df_obs_o3_6mDM8h_ppb.csv')

In [ ]:
df_obs['2017']

In [ ]:
df_obs.loc[df_obs.station_id == '1001A']